# Chương trình tiền xử lý dữ liệu

## Giới thiệu

- **Tên chương trình:** Honoda (**Hono**ka's **Da**ta Preprocessor), được đặt tên theo nhân vật anime yêu thích của em - Kousaka Honoka `=)`.
- **Ngôn ngữ lập trình:** Python 3.9.5.
- **Các thư viện đã sử dụng:**
    - `argparse`: thư viện phân tích tham số dòng lệnh của Python.
    - `csv`: thư viện đọc/ghi file CSV của Python.
    - `re`: thư viện xử lý biểu thức chính quy. Dùng để phân tích pattern của dữ liệu để đoán kiểu dữ liệu (do data đọc vào từ CSV là dạng chuỗi).
    - `copy`: để copy các object trong Python, tránh trường hợp các mutable member variables bị cross-referenced.
    - `math`: các hàm toán học trong Python.
    - `operator`: thư viện để lấy ra các tham chiếu đến các toán tử cơ bản trong Python (như `add`, `sub`, ...) để implement operator overloading.
    - `honolib`: thư viện xử lý dataframe cơ bản do em phát triển.
    - Chương trình không sử dụng các thư viện ngoài.  
    
Báo cáo của bài này em viết bằng Jupyter Notebook, có đính kèm theo notebook gốc trước khi convert sang PDF nên thầy có thể thử chạy để kiểm chứng kết quả ạ. 

Trong quá trình viết báo cáo, em sẽ sử dụng thư viện `pandas` để **đối chiếu kết quả** thực hiện của 2 bên trong một số trường hợp để xem implementation của em có hoạt động "chuẩn" chưa. Ngoài ra có một số chức năng em cần cho quá trình demo mà đề không yêu cầu (như xuất ra 10 dòng đầu của file thay vì xuất hết 1000 dòng) thì em xin được dùng một số câu lệnh trên terminal của Linux (như `head`, `tail`, `grep`, ...) để tiết kiệm thời gian ạ.

## Danh sách các lớp của chương trình
![class-diagram](images/class_diagram.png)

- `Series`: biểu diễn một cột của bảng. 
- `DataFrame`: biểu diễn một bảng. 
- `SeriesExpression`: biểu diễn một biểu thức mà các toán hạng là các cột.
- `Controller`: để gom nhóm các hàm xử lý của chương trình lại. Mỗi phương thức ứng với một chức năng yêu cầu của đề. Hàm main sẽ phân tích tham số dòng lệnh để gọi đến các phương thức tương ứng trong class này.

Trên đây là sơ đồ lớp các lớp chính của chương trình do em đã thiết kế. Do số lượng hàm khá nhiều nên em cũng xin không ghi các hàm private cũng như các hàm liên quan đến quá tải toán tử trong này. Để biết chức năng và cách sử dụng mỗi hàm, thầy có thể xem source code của các module tương ứng. Mỗi hàm em đều có viết một docstring để mô tả tham số, chức năng và dữ liệu trả về.

## Báo cáo các chức năng chính

Cơ bản em đã hoàn thiện được tất cả 8 chức năng mà đề yêu cầu. Cụ thể như các mục con ở bên dưới.

Thử gọi lệnh help của chương trình:

In [1]:
%cd ../src

/home/hiraki/source/DataMining/DataCleasing/src


In [2]:
!python3 honoda.py --help

usage: honoda.py [-h] [-o OUTPUT] [-v]
                 {stats,fillna,dropna,dropdup,normalize,evaluate} ... INPUT

Honoka's Data Preprocessing Toolbox

positional arguments:
  {stats,fillna,dropna,dropdup,normalize,evaluate}
                        Honoda subcommands
    stats               Display data statistics
    fillna              Fill null values
    dropna              Drop column or row with null values
    dropdup             Only keep unique rows from the dataframe
    normalize           Normalize a column
    evaluate            Evaluate a columnar expression
  INPUT                 Input filename

optional arguments:
  -h, --help            show this help message and exit
  -o OUTPUT, --out OUTPUT
                        Output filename
  -v, --verbose         Increase verbosity


In [3]:
!python3 honoda.py stats --help

usage: honoda.py stats [-h] [--shape | -ls | -d COLUMN | -cnr | -lnc]

optional arguments:
  -h, --help            show this help message and exit
  --shape               Print dataframe's shape
  -ls, --list-columns   List all columns and their detected datatype
  -d COLUMN, --describe COLUMN
                        Display statistical info (min, max, mean, mode, ...)
                        of a column
  -cnr, --count-null-rows
                        Count all rows with at least one null value, and
                        display the total null-ed rows
  -lnc, --list-null-columns
                        List all columns with at least one null value, and
                        their total null counts


### Thử liệt kê ra 10 cột đầu và cuối của bảng

In [4]:
# Liệt kê tất cả các cột, ghi ra file cols.txt
!python3 honoda.py stats --list-columns ../house-prices.csv > cols.txt

# Xuất 10 cột đầu và 10 cột cuối của bảng (dùng lệnh head và tail của Linux).
!head -n 10 cols.txt 
!echo
!tail -n 10 cols.txt
!rm cols.txt

Id              	 <class 'int'>
MSSubClass      	 <class 'int'>
MSZoning        	 <class 'str'>
LotFrontage     	 <class 'float'>
LotArea         	 <class 'int'>
Street          	 <class 'str'>
Alley           	 <class 'str'>
LotShape        	 <class 'str'>
LandContour     	 <class 'str'>
Utilities       	 <class 'str'>

PoolQC          	 None
Fence           	 <class 'str'>
MiscFeature     	 <class 'str'>
MiscVal         	 <class 'int'>
MoSold          	 <class 'int'>
YrSold          	 <class 'int'>
SaleType        	 <class 'str'>
SaleCondition   	 <class 'str'>
SalePrice       	 <class 'int'>
Total: 81 columns.


Cột `PoolQC` có kiểu là `None` vì cột này không chứa bất cứ dữ liệu nào.

### Yêu cầu 1: Liệt kê các cột bị thiếu dữ liệu

In [5]:
!python3 honoda.py stats --list-null-columns ../house-prices.csv

LotFrontage   	 173
Alley         	 941
MasVnrType    	 593
MasVnrArea    	 10
BsmtQual      	 27
BsmtCond      	 27
BsmtExposure  	 28
BsmtFinType1  	 27
BsmtFinType2  	 29
FireplaceQu   	 501
GarageType    	 60
GarageYrBlt   	 60
GarageFinish  	 60
GarageQual    	 60
GarageCond    	 60
PoolQC        	 1000
Fence         	 815
MiscFeature   	 963

Total null values: 5434


Thử kiểm tra lại bằng thư viện "chuẩn" `pandas`:

In [6]:
import pandas as pd
df = pd.read_csv('../house-prices.csv')

In [7]:
for col in df.columns:
    null_val = df[col].isnull().sum()
    if null_val > 0:
        print(col.ljust(20), null_val)

LotFrontage          173
Alley                941
MasVnrType           593
MasVnrArea           10
BsmtQual             27
BsmtCond             27
BsmtExposure         28
BsmtFinType1         27
BsmtFinType2         29
FireplaceQu          501
GarageType           60
GarageYrBlt          60
GarageFinish         60
GarageQual           60
GarageCond           60
PoolQC               1000
Fence                815
MiscFeature          963


### Yêu cầu 2: Đếm số hàng bị thiếu dữ liệu

In [8]:
!python3 honoda.py stats --count-null-rows ../house-prices.csv

Number of null rows in dataframe: 1000


Kiểm tra lại bằng `pandas`:

In [9]:
len(df) - len(df.dropna())

1000

### Yêu cầu 3: Điền giá trị thiếu với thuộc tính numeric (phương pháp mean)
Bây giờ ta thử điền giá trị thiếu cho cột `LotFrontage`. Theo như ta đã thấy ở trên thì cột này có kiểu dữ liệu là `int`.
2 câu lệnh bên dưới để in ra thông số của cột `LotFrontage` trước khi thực hiện fill, và sau đó gọi lệnh để fill các giá trị null bằng phương pháp mean, sau đó xuất ra file `out.csv`. Câu lệnh cuối cùng để kiểm tra lại kết quả sau khi fill.

In [10]:
# Xem thông số cột trước khi fill
!python3 honoda.py stats --describe LotFrontage ../house-prices.csv

Type:             <class 'float'>
Length:           1000
Null values:      173
Label:            LotFrontage
Sum:              57314.0
Mean:             69.3035
Median:           73.0
Std:              21.2598
Min-Max:          (21.0, 153.0)
Mode:             (60.0, 117)


In [11]:
# Fill với phương pháp mean
!python3 honoda.py -o out.csv fillna --method mean --column LotFrontage ../house-prices.csv
# In lại thông số của cột sau khi fill
!python3 honoda.py stats --describe LotFrontage out.csv

Type:             <class 'float'>
Length:           1000
Null values:      0
Label:            LotFrontage
Sum:              69303.5067
Mean:             69.3035
Median:           69.3035
Std:              19.3336
Min-Max:          (21.0, 153.0)
Mode:             (69.30350665054414, 173)


Ta nhận thấy số giá trị null đã giảm về 0. Ngoài ra, giá trị mean là 69.3035 do bị điền lặp đi lặp lại tới 173 lần cũng đã thay thế 60.0, trở thành giá trị mode của cột.

Thử lại với cột `PoolQC` (cột hoàn toàn rỗng, không chứa dữ liệu).

In [12]:
!echo "Before"
!python3 honoda.py stats --describe PoolQC ../house-prices.csv
!python3 honoda.py -o out.csv fillna --method mean --column PoolQC ../house-prices.csv
!echo "\nAfter"
!python3 honoda.py stats --describe PoolQC out.csv

Before
Type:             None
Length:           1000
Null values:      1000
Label:            PoolQC
Sum:              None
Mean:             None
Median:           None
Std:              None
Min-Max:          None
Mode:             (0, 0)
Forcing method "mode" for categorical column

After
Type:             <class 'int'>
Length:           1000
Null values:      0
Label:            PoolQC
Sum:              0
Mean:             0.0
Median:           0
Std:              0.0
Min-Max:          (0, 0)
Mode:             (0, 1000)


Trong tham số dòng lệnh mặc dù em đã cho `--method=mean` nhưng chương trình vẫn fill bằng phương pháp `mode`, là vì nó đã nhận diện cột này là categorical (cột rỗng tính là categorical) và bắt buộc phải fill theo `mode` thay vì `mean`.  
Ngoài ra, dễ thấy được cột đã được fill bằng giá trị `0`. Điều này là nếu do cột rỗng thì biến đếm của cột (trong implementation) sẽ không bao giờ tăng nên nó vẫn ở giá trị `0`, do đó cột được fill bằng giá trị `0`.

### Yêu cầu 3: Điền giá trị thiếu với thuộc tính numeric (phương pháp median)
Tương tự như trên, nhưng lần này ta thực hiện với cột `GarageYrBlt`. Cột này có kiểu dữ liệu `float`.

In [13]:
# In ra thông số cột trước khi fill
!python3 honoda.py stats --describe GarageYrBlt ../house-prices.csv

Type:             <class 'float'>
Length:           1000
Null values:      60
Label:            GarageYrBlt
Sum:              1859474.0
Mean:             1978.1638
Median:           1985.0
Std:              25.2207
Min-Max:          (1900.0, 2010.0)
Mode:             (2006.0, 42)


In [14]:
# Fill cột với phương pháp median
!python3 honoda.py -o out.csv fillna --method median --column GarageYrBlt ../house-prices.csv
# In lại thông số cột sau khi fill
!python3 honoda.py stats --describe GarageYrBlt out.csv

Type:             <class 'float'>
Length:           1000
Null values:      0
Label:            GarageYrBlt
Sum:              1978574.0
Mean:             1978.574
Median:           1985.0
Std:              24.5062
Min-Max:          (1900.0, 2010.0)
Mode:             (1985.0, 65)


### Yêu cầu 3: Điền giá trị thiếu với thuộc tính categorical (phương pháp mode)
Tương tự như trên, nhưng lần này ta thực hiện với cột `GarageType`. Cột này có kiểu dữ liệu `str`.

In [15]:
# In ra thông số cột trước khi fill
!python3 honoda.py stats --describe GarageType ../house-prices.csv

Type:             <class 'str'>
Length:           1000
Null values:      60
Label:            GarageType
Sum:              None
Mean:             None
Median:           None
Std:              None
Min-Max:          None
Mode:             ('Attchd', 572)


In [16]:
# Fill cột với phương pháp mode
!python3 honoda.py -o out.csv fillna --column=GarageType --method=mode ../house-prices.csv
# In lại thông số cột sau khi fill
!python3 honoda.py stats --describe GarageType out.csv

Forcing method "mode" for categorical column
Type:             <class 'str'>
Length:           1000
Null values:      0
Label:            GarageType
Sum:              None
Mean:             None
Median:           None
Std:              None
Min-Max:          None
Mode:             ('Attchd', 632)


Nếu để ý, ta sẽ thấy được trước khi fill, mode của cột này là `Attchd`, và có tần suất xuất hiện là 572. Cột có 60 giá trị null. Sau khi fill, mode vẫn giữ nguyên, và tần suất xuất hiện tăng lên một lượng đúng bằng 60 (572 + 60 = 632). 

Thử lại với cột `FireplaceQu`, ta cũng quan sát được điều tương tự:

In [17]:
# In ra thông số cột trước khi fill
!python3 honoda.py stats --describe FireplaceQu ../house-prices.csv

Type:             <class 'str'>
Length:           1000
Null values:      501
Label:            FireplaceQu
Sum:              None
Mean:             None
Median:           None
Std:              None
Min-Max:          None
Mode:             ('Gd', 242)


In [18]:
# Fill cột với phương pháp mode
!python3 honoda.py -o out.csv fillna --column=FireplaceQu --method=mode ../house-prices.csv
# In lại thông số cột sau khi fill
!python3 honoda.py stats --describe FireplaceQu out.csv

Forcing method "mode" for categorical column
Type:             <class 'str'>
Length:           1000
Null values:      0
Label:            FireplaceQu
Sum:              None
Mean:             None
Median:           None
Std:              None
Min-Max:          None
Mode:             ('Gd', 743)


### Yêu cầu 4: Xóa các cột bị thiếu với ngưỡng tỉ lệ thiếu cho trước

Giả sử ta chọn `threshold = 0.6`. Ta nhận thấy bảng có 1000 dòng, mà $0.6 \times 1000 = 600$, nghĩa là những cột nào có nhiều hơn hoặc bằng 600 giá trị null sẽ bị "bay màu".  
Dựa vào bảng thống kê ở phần *Đếm số hàng bị thiếu dữ liệu*, ta thấy các cột `Alley`, `Fence`, `PoolQC`, `MiscFeature` sẽ bị drop nếu ta chọn threshold này.

In [19]:
!python3 honoda.py -o out.csv dropna --axis=1 --threshold=0.6 ../house-prices.csv

Deleted "Alley"; Null = 94.1%
Deleted "PoolQC"; Null = 100.0%
Deleted "Fence"; Null = 81.5%
Deleted "MiscFeature"; Null = 96.3%


Đúng như dự đoán, các cột nêu trên đã bị xóa. Để chắc chắn hơn, bây giờ ta thử đọc file output lên để xem các cột đó còn không.

In [20]:
!python3 honoda.py stats -lnc out.csv

LotFrontage   	 173
MasVnrType    	 593
MasVnrArea    	 10
BsmtQual      	 27
BsmtCond      	 27
BsmtExposure  	 28
BsmtFinType1  	 27
BsmtFinType2  	 29
FireplaceQu   	 501
GarageType    	 60
GarageYrBlt   	 60
GarageFinish  	 60
GarageQual    	 60
GarageCond    	 60

Total null values: 1715


Thử lại với `threshold = 0.5`. Ta có $0.5 \times 1000 = 500$, nên tương tự, các cột `Alley`, `Fence`, `PoolQC`, `MiscFeature`, `FireplaceQu`, `MasVnrType` sẽ bị xóa.

In [21]:
!python3 honoda.py -o out.csv dropna --axis=1 --threshold=0.5 ../house-prices.csv

Deleted "Alley"; Null = 94.1%
Deleted "MasVnrType"; Null = 59.3%
Deleted "FireplaceQu"; Null = 50.1%
Deleted "PoolQC"; Null = 100.0%
Deleted "Fence"; Null = 81.5%
Deleted "MiscFeature"; Null = 96.3%


### Yêu cầu 5: Xóa các hàng bị thiếu với ngưỡng tỉ lệ thiếu cho trước
In ra lại các hàng bị thiếu dữ liệu

In [22]:
!python3 honoda.py stats -cnr ../house-prices.csv

Number of null rows in dataframe: 1000


Ta thử drop các hàng bị thiếu dữ liệu với threshold = 0.07. Bảng có 81 cột, vậy nếu hàng nào có $ceil(81 \times 0.07) = 6$ cột bị thiếu sẽ bị drop.

In [48]:
!python3 honoda.py -o out.csv dropna --axis=0 --threshold=0.07 ../house-prices.csv

In [24]:
!python3 honoda.py stats -cnr out.csv

Number of null rows in dataframe: 254


Thử sử dụng `pandas` để kiểm chứng

In [25]:
print('Before:', len(df) -  len(df.dropna()))
df1 = df.dropna(thresh=81 * (1 - 0.07))
print('After:', len(df1) -  len(df1.dropna()))

Before: 1000
After: 656


Thử lại với với threshold = 0.05. Bảng có 81 cột, vậy nếu hàng nào có $ceil(81 \times 0.05) = 5$ cột bị thiếu sẽ bị drop.

In [49]:
# Số hàng ban đầu
print('Before drop:')
!python3 honoda.py stats -cnr ../house-prices.csv
!python3 honoda.py -o out.csv dropna --axis=0 --threshold=0.05 ../house-prices.csv
print('After drop:')
!python3 honoda.py stats -cnr out.csv

Before drop:
Number of null rows in dataframe: 1000
After drop:
Number of null rows in dataframe: 307


Thử sử dụng `pandas` để kiểm chứng

In [27]:
print('Before:', len(df) -  len(df.dropna()))
df1 = df.dropna(thresh=81 * (1 - 0.05))
print('After:', len(df1) -  len(df1.dropna()))

Before: 1000
After: 307


### Yêu cầu 6: Xóa các mẫu bị trùng lặp
Ta in ra thông tin của bảng trước khi xóa

In [28]:
!python3 honoda.py stats --shape ../house-prices.csv

(1000, 81)


In [29]:
!python3 honoda.py -o out.csv dropdup ../house-prices.csv

In [30]:
# In ra thông tin sau khi xóa
!python3 honoda.py stats --shape out.csv

(716, 81)


Vậy có 284 mẫu trùng lặp đã bị xóa.  
Ta thử kiểm tra lại bằng pandas:

In [31]:
print('Before:', df.shape)
df1 = df.drop_duplicates()
print('After:', df1.shape)

Before: (1000, 81)
After: (716, 81)


\newcommand{\xx}{\mathbf{x}}
### Yêu cầu 7: Chuẩn hóa một cột (phương pháp z-score)
Gọi vector cần chuẩn hóa là $\xx$. Công thức để chuẩn hóa theo phương pháp z-score:

$$
\xx = \frac{\xx - \mu}{\sigma}
$$

Trong phần này em sẽ lấy cột `GarageYrBlt` để thử.

In [32]:
# In ra thông tin cột GarageYrBlt
!python3 honoda.py stats --describe GarageYrBlt ../house-prices.csv

Type:             <class 'float'>
Length:           1000
Null values:      60
Label:            GarageYrBlt
Sum:              1859474.0
Mean:             1978.1638
Median:           1985.0
Std:              25.2207
Min-Max:          (1900.0, 2010.0)
Mode:             (2006.0, 42)


In [33]:
# Chuẩn hóa bằng z-score 
!python3 honoda.py -o out.csv normalize --method zscore --column GarageYrBlt ../house-prices.csv

In [34]:
# Đọc lại thông tin của cột sau khi được chuẩn hóa
!python3 honoda.py stats --describe GarageYrBlt out.csv

Type:             <class 'float'>
Length:           1000
Null values:      60
Label:            GarageYrBlt
Sum:              -0.0
Mean:             -0.0
Median:           0.2711
Std:              1.0
Min-Max:          (-3.0991932928723527, 1.2623031069833908)
Mode:             (1.1037032378977272, 42)


Vậy sau khi chuẩn hóa bằng phương pháp z-score, độ lệnh chuẩn và trung bình của cột đã được đưa về giá trị $\mu=0, \sigma=1$.

Ta thử lại với cột `TotRmsAbvGrd`:

In [35]:
# In ra thông tin cột TotRmsAbvGrd
print('Before:')
!python3 honoda.py stats --describe TotRmsAbvGrd ../house-prices.csv
# Chuẩn hóa bằng z-score 
!python3 honoda.py -o out.csv normalize --method zscore --column TotRmsAbvGrd ../house-prices.csv
# Đọc lại thông tin của cột sau khi được chuẩn hóa
print('\nAfter:')
!python3 honoda.py stats --describe TotRmsAbvGrd out.csv

Before:
Type:             <class 'int'>
Length:           1000
Null values:      0
Label:            TotRmsAbvGrd
Sum:              6505
Mean:             6.505
Median:           6
Std:              1.6426
Min-Max:          (2, 12)
Mode:             (6, 289)

After:
Type:             <class 'float'>
Length:           1000
Null values:      0
Label:            TotRmsAbvGrd
Sum:              0.0
Mean:             0.0
Median:           -0.3074
Std:              1.0
Min-Max:          (-2.7426843901867493, 3.3454052661656353)
Mode:             (-0.3074485276457954, 289)


Ta cũng quan sát được sau khi chuẩn hóa, $\mu=0$ và $\sigma=1$.

### Yêu cầu 7: Chuẩn hóa một cột (phương pháp min-max)
Để chuẩn hóa vector $\xx$ theo phương pháp min-max, ta sử dụng công thức:
$$ 
\xx = \frac{\xx - min(\xx)}{max(\xx) - min(\xx)}
$$

Trong phần này em sẽ lấy cột `SalePrice` để thử.

In [36]:
# In ra thông tin cột SalePrice
!python3 honoda.py stats --describe SalePrice ../house-prices.csv

Type:             <class 'int'>
Length:           1000
Null values:      0
Label:            SalePrice
Sum:              178116040
Mean:             178116.04
Median:           159000
Std:              80133.889
Min-Max:          (35311, 611657)
Mode:             (135000, 16)


In [37]:
# Chuẩn hóa bằng min-max
!python3 honoda.py -o out.csv normalize --method minmax --column SalePrice ../house-prices.csv

In [38]:
# Đọc lại thông tin của cột sau khi được chuẩn hóa
!python3 honoda.py stats --describe SalePrice out.csv

Type:             <class 'float'>
Length:           1000
Null values:      0
Label:            SalePrice
Sum:              247.7766
Mean:             0.2478
Median:           0.2146
Std:              0.139
Min-Max:          (0.0, 1.0)
Mode:             (0.17296728007134604, 16)


Vậy sau khi chuẩn hóa bằng phương pháp min-max, miền giá trị của cột đã được đưa về $(0, 1)$ (có thể nhìn thấy qua giá trị của trường `Min-Max`).

Ta thử lại với cột `OpenPorchSF`:

In [39]:
# In ra thông tin cột PoolArea
print('Before:')
!python3 honoda.py stats --describe OpenPorchSF ../house-prices.csv
# Chuẩn hóa bằng min-max
!python3 honoda.py -o out.csv normalize --method minmax --column OpenPorchSF ../house-prices.csv
# Đọc lại thông tin của cột sau khi được chuẩn hóa
print('\nAfter:')
!python3 honoda.py stats --describe OpenPorchSF out.csv

Before:
Type:             <class 'int'>
Length:           1000
Null values:      0
Label:            OpenPorchSF
Sum:              44870
Mean:             44.87
Median:           20
Std:              66.3318
Min-Max:          (0, 547)
Mode:             (0, 476)

After:
Type:             <class 'float'>
Length:           1000
Null values:      0
Label:            OpenPorchSF
Sum:              82.0293
Mean:             0.082
Median:           0.0366
Std:              0.1213
Min-Max:          (0.0, 1.0)
Mode:             (0.0, 476)


Ta cũng quan sát được dữ liệu của cột đã được đưa về khoảng $(0, 1)$.

### Yêu cầu 8: Tính giá trị biểu thức thuộc tính

Ở phần này, em đã cài đặt thuật toán shunting yard (biến đổi về suffix expression rồi evaluate) nên biểu thức có quy định ngặt nghèo một chút xíu (để dễ code hơn). Ngoài ra em cũng không check syntax error (vì trong đề có nói *bạn không cần phải kiểm tra tính hợp lệ của dữ liệu*). 

- Các toán tử được phép sử dụng: +, -, *, /. Được phép sử dụng dấu ngoặc đơn `()`.
- Các token phải cách nhau bằng khoảng trắng. Vd: `LotFrontage + YrSold / 2`.
- Nếu thực hiện phép tính giữa cột và số thực: số phải đứng sau cột. Vd: `YrSold / 2`, `LotFrontage + 2.5`.

 

**Giả sử** ta muốn tính thuế VAT (10%) của một $m^2$ đất khi mua căn nhà có giá trị bao nhiêu (giả sử cột `SalePrices` chưa có VAT), ta cần tính giá trị của biểu thức:   

$$
\frac{SalePrice}{LotArea} \times 0.1
$$

Công thức trên chỉ là giả sử vì em chưa mua nhà bao giờ nên không biết có chính xác không `=)`.

In [40]:
!python3 honoda.py -o out.csv evaluate "( SalePrice / LotArea ) * 0.1" ../house-prices.csv

In [41]:
# Liệt kê ra 5 dòng đầu của cột kết quả
!head -n 6 out.csv

out
2.5213524215656413
1.0343426132899818
2.0
1.4462809917355373
1.128632033939006


Thử in ra các tham số thống kê của cột mới này

In [42]:
!python3 honoda.py stats --describe out out.csv

Type:             <class 'float'>
Length:           1000
Null values:      0
Label:            out
Sum:              2127.1049
Mean:             2.1271
Median:           1.7877
Std:              1.2745
Min-Max:          (0.17422007479848545, 8.384506376948513)
Mode:             (1.581798483206934, 4)


Thử kiểm chứng lại kết quả bằng `pandas`:

In [43]:
result = (df['SalePrice'] / df['LotArea']) * 0.1
result.head(5)

0    2.521352
1    1.034343
2    2.000000
3    1.446281
4    1.128632
dtype: float64

In [44]:
result.describe()

count    1000.000000
mean        2.127105
std         1.275179
min         0.174220
25%         1.377803
50%         1.787686
75%         2.458818
max         8.384506
dtype: float64

Quan sát 5 dòng đầu và 2 thông số `mean`, `std`, ta có thể thấy được các kết quả hoàn toàn trùng khớp với nhau.

Thử lại với một biểu thức khác. Giả sử lần này em muốn tóm tắt thông tin của ngói nhà nên em muốn join 2 cột **text** là `RoofStyle` và `RoofMatl`. Lý do cái này hoạt động được là vì ở bên dưới em implement bằng operator overloading nên hễ kiểu dữ liệu của 2 cột (trong Python) có hỗ trợ các toán tử đã nêu thì đều có thể thực hiện được phép tính.

In [45]:
!python3 honoda.py -o out.csv evaluate "RoofStyle + RoofMatl " ../house-prices.csv

In [46]:
!head -n 11 out.csv

out
HipCompShg
GableCompShg
GableCompShg
GableCompShg
GableCompShg
GableCompShg
GableCompShg
HipCompShg
GableCompShg
GableCompShg


Kiểm tra lại bằng `pandas`:

In [47]:
(df['RoofStyle'] + df['RoofMatl']).head(10)

0      HipCompShg
1    GableCompShg
2    GableCompShg
3    GableCompShg
4    GableCompShg
5    GableCompShg
6    GableCompShg
7      HipCompShg
8    GableCompShg
9    GableCompShg
dtype: object

Ta cũng có thể thấy được 2 kết quả hoàn toàn trùng khớp với nhau.

# Tài liệu tham khảo 
1. Slide bài giảng lý thuyết.
2. Pandas documentation: \url{https://pandas.pydata.org/docs/}
3. Python documentation -- `argparse`: \url{https://docs.python.org/3/library/argparse.html}
4. Python documentation -- `csv`: \url{https://docs.python.org/3/library/csv.html}
5. GeeksforGeeks: Infix to Postfix conversion: \url{https://www.geeksforgeeks.org/stack-set-2-infix-to-postfix/}